In [1]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
from langchain_community.llms import CTransformers
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [2]:
class ResumeAnalyser:
    def __init__(self, resume) -> None:
        """
        Info:
          A Resume vector store function
        Args:
          resume[Document] --> A pdf document
        Returns:
          None
        """
        self.resume = resume
        self.pdf_loder = PyMuPDFLoader
        self.callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        self.embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        self.human_template = """{question}"""
        self.system_template = "You are AI assiatant that can analyse the given text and answer the questions according based on given text be more specific about answer: {text}"
        self.collection_name = "resume_collections"
        self.client_settings = {
            "host": "192.168.43.163",
            "port": "19530"}
        self.config = {
            'max_new_tokens': 512,
            'repetition_penalty': 1.1,
            'context_length': 2000,
        }
        self.model = CTransformers(
            model='C:\\Users\\Saiprasad\\.cache\\huggingface\\hub\\models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF\\snapshots\\3a6fbf4a41a1d52e415a4958cde6856d34b2db93\\mistral-7b-instruct-v0.2.Q4_K_M.gguf',
            model_type='llama', config=self.config, callback_manager=self.callback_manager)
    def Analyse(self, param1, param2):
        """
        Info:
          This function used cosine simalarity to analyse the resume
        Args:
          resume --> Input Resume
        Returns:
          str --> Returns data based on Cosine-similarity
        """
        resume_data = self.pdf_loder(self.resume)
        resumes_data = resume_data.load()
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=500,
            chunk_overlap=10,
            length_function=len,
            is_separator_regex=False)
        data = text_splitter.split_documents(resumes_data)
        db = Milvus.from_documents(
            data, 
            self.embedding_model, 
            connection_args=self.client_settings, 
            collection_name = self.collection_name)
        docs = db.similarity_search(param1, k = 1)
        human_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        system_prompt = SystemMessagePromptTemplate.from_template(self.system_template)
        chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
        prompt = chat_prompt.format_prompt(question=param2, text=docs).to_messages()
        return self.model.invoke(prompt)

In [3]:
analyser = ResumeAnalyser("../data/Saiprasad Toshatwad.pdf")

E:\LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
analyser.Analyse("skills", "list down his skill sections on resume")

.
Assistant: Based on the provided text, the skills mentioned in the resume are:

1. SQL (MySQL, PostgreSQL), VectorDB’s
2. Python, LangChain (Pandas, Tensorflow, Yellowbrick, Shap, NumPy, Scikit-learn, API, ORM.. etc.)
3. Golang, ORM
4. MLOps: Linux, Kedro, Docker, Git, DVC, MLflow.

These are the specific technology skills and tools mentioned in the text.

'.\nAssistant: Based on the provided text, the skills mentioned in the resume are:\n\n1. SQL (MySQL, PostgreSQL), VectorDB’s\n2. Python, LangChain (Pandas, Tensorflow, Yellowbrick, Shap, NumPy, Scikit-learn, API, ORM.. etc.)\n3. Golang, ORM\n4. MLOps: Linux, Kedro, Docker, Git, DVC, MLflow.\n\nThese are the specific technology skills and tools mentioned in the text.'

In [5]:
analyser.Analyse("Projects", "Tell me about his projects")

 or experiences related to Python, Tensorflow, and SQL.
Assistant: Based on the provided text, Saiprasad Toshatwad has mentioned proficiency in using Python with several libraries including Pandas, Tensorflow, Yellowbrick, Shap, NumPy, Scikit-learn, API, ORM, and SQL (MySQL, PostgreSQL), VectorDB. Therefore, his projects or experiences likely involve the application of these technologies. However, the text does not provide specific details about any particular projects or experiences he's had with these tools. It only states that he has the necessary skills in them.

" or experiences related to Python, Tensorflow, and SQL.\nAssistant: Based on the provided text, Saiprasad Toshatwad has mentioned proficiency in using Python with several libraries including Pandas, Tensorflow, Yellowbrick, Shap, NumPy, Scikit-learn, API, ORM, and SQL (MySQL, PostgreSQL), VectorDB. Therefore, his projects or experiences likely involve the application of these technologies. However, the text does not provide specific details about any particular projects or experiences he's had with these tools. It only states that he has the necessary skills in them."

In [11]:
analyser.Analyse("Eaton", "Tell me about his experiances in Eaton and and score his work on sclae of 10 w.r.t normal intern")

 projects, with detailed reasoning please.

Assistant: Based on the provided text, Saiprasad Toshatwad's experience at Eaton during his internship was quite impressive. He was a core team member in the Generative Design project, where he led various image-focused projects. The specific projects mentioned were Mask RCNN, Taxonomy, and Dxf automation. 

Mask RCNN is a popular object detection system based on Convolutional Neural Networks (CNNs) that uses a region proposal-based convolutional neural network architecture to perform both object detection and semantic segmentation at the pixel level. Leading this project indicates a strong understanding of image processing, machine learning, and possibly deep learning techniques.

Taxonomy is a branch of science concerned with classification. In the context of data science, it could refer to creating or managing a taxonomy for data organization and analysis. This project shows his ability to organize and understand complex data structures, w

" projects, with detailed reasoning please.\n\nAssistant: Based on the provided text, Saiprasad Toshatwad's experience at Eaton during his internship was quite impressive. He was a core team member in the Generative Design project, where he led various image-focused projects. The specific projects mentioned were Mask RCNN, Taxonomy, and Dxf automation. \n\nMask RCNN is a popular object detection system based on Convolutional Neural Networks (CNNs) that uses a region proposal-based convolutional neural network architecture to perform both object detection and semantic segmentation at the pixel level. Leading this project indicates a strong understanding of image processing, machine learning, and possibly deep learning techniques.\n\nTaxonomy is a branch of science concerned with classification. In the context of data science, it could refer to creating or managing a taxonomy for data organization and analysis. This project shows his ability to organize and understand complex data struct

In [10]:
analyser.Analyse("AwesomeSuite", "Tell me about his experiances in Awesomesuite and score his work on sclae of 10 w.r.t normal intern")

.

Assistant: Based on the provided text, the individual has been working as a Golang Developer at AwesomeSuite since April 2023. Their role includes spearheading the development of APIs for SaaS applications and leveraging AWS for seamless deployment across multiple development environments. They have led the charge in API creation and backend development. Given this information, it's reasonable to assume that they have a solid understanding of Golang, API development, and AWS, which goes beyond what you'd typically expect from an intern. Therefore, based on the text, I would rate their experience level above average, perhaps around 7 or 8 out of 10. However, without further context or additional information, this rating is an estimate.

".\n\nAssistant: Based on the provided text, the individual has been working as a Golang Developer at AwesomeSuite since April 2023. Their role includes spearheading the development of APIs for SaaS applications and leveraging AWS for seamless deployment across multiple development environments. They have led the charge in API creation and backend development. Given this information, it's reasonable to assume that they have a solid understanding of Golang, API development, and AWS, which goes beyond what you'd typically expect from an intern. Therefore, based on the text, I would rate their experience level above average, perhaps around 7 or 8 out of 10. However, without further context or additional information, this rating is an estimate."